In [1]:
import ast
import json
from bs4 import BeautifulSoup
from transformers import pipeline
summarizer = pipeline("summarization")
import re
import pandas as pd
import numpy as np
import os
os.chdir(r'..\data')
from keybert import KeyBERT
kw_model = KeyBERT()
from tqdm import tqdm
tqdm.pandas()
from joblib import Parallel, delayed
import multiprocessing
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Opening JSON file
f = open('newsSample.json', "r", encoding='utf-8')
data = json.load(f, strict=False)

In [3]:
len(data)

1000

In [4]:
data[45]

{'newsId': 759904,
 'vendorId': 25998310,
 'title': 'Digital Euro Coming? 13 Things We Know About the ECB?s Potential Plans',
 'excerpt': 'The <strong>European Central Bank</strong> (ECB) is working on a digital euro but it will still',
 'body': '<p>The European Central Bank (ECB) is working on a digital euro but it will still be some time before the virtual currency will see use.</p><p>Source: peterschreiber.media/Shutterstock.com </p><p>Let’s take a look at everything investors need to know about the digital euro and ECB’s possible plans.</p><p>The ECB will likely be setting up a wallet to hold the digital euro for citizens. That could include holding limits to keep people from abandoning commercial banks. This would likely be similar to how a digital bank account works. However, a major difference is that the ECB has no worries about running out of funds. One thing to note is that there’s no guarantee that the ECB will use blockchain for the digital currency. That seems like a stran

In [5]:
doc_df = pd.DataFrame(data, columns=['json_doc'])
doc_df.head(2)

,json_doc
0,NaN
1,NaN


In [6]:
doc_index=['doc_'+str(idx) for idx, json_doc in enumerate(data)]
len(doc_index), len(data)

(1000, 1000)

In [7]:
doc_df = pd.DataFrame({'doc_index':doc_index,'doc':data})
doc_df = doc_df.sample(50, random_state=50)
doc_df = doc_df.reset_index()
def get_document_text(jdoc):
    title = jdoc['title']
    excerpt = jdoc['excerpt']
    body = jdoc['body']
    combined_text = title +'\n'+excerpt+'\n'+body
    soup_combined_text = BeautifulSoup(combined_text)
    clean_combined_text = soup_combined_text.get_text()
    return clean_combined_text

def get_top_n_phrases(jdoc, n, colls):
    doctext = get_document_text(jdoc)
    keywords = kw_model.extract_keywords(doctext, keyphrase_ngram_range=(2, 3), stop_words='english', top_n=n)
    keywords_df = pd.DataFrame(keywords, columns=colls)
    return keywords_df

colls = ['phrase', 'importance']
#doc_df['extraction'] = doc_df['doc'].progress_apply(get_top_n_phrases, n=5, colls=colls)
#doc_df['extraction'] = doc_df['doc'].apply(get_top_n_phrases, n=5, colls=colls)
process_doc = [get_top_n_phrases(doc_, n=5, colls=colls)  for doc_ in doc_df['doc'].tolist()]
doc_df['doc_phrases'] = pd.Series(process_doc)
doc_df.head()


,index,doc_index,doc,doc_phrases
0,316,doc_316,"{'newsId': 759811, 'vendorId': 25997375, 'titl...",phrase importance 0 wipr...
1,459,doc_459,"{'newsId': 761637, 'vendorId': 26000979, 'titl...",phrase importance 0 ...
2,854,doc_854,"{'newsId': 757053, 'vendorId': 25989833, 'titl...",phrase importance 0 ...
3,11,doc_11,"{'newsId': 761689, 'vendorId': 26001229, 'titl...",phrase importance 0...
4,978,doc_978,"{'newsId': 756889, 'vendorId': 25989405, 'titl...",phrase importance 0 l...


In [8]:
doc_dict = dict(zip(doc_df['doc_index'],doc_df['doc']))
with open('doc_data_10aug2021.json', 'w') as fp:
    json.dump(doc_dict, fp)